#### Import Libraries

In [0]:
from databricks import feature_store
from databricks.feature_store import feature_table

#### Read Parameters of feature store

In [0]:
src_table = dbutils.widgets.get("src_table")

feature_store_table = dbutils.widgets.get("feature_store_table")

feature_store_primary_keys = dbutils.widgets.get("feature_store_primary_keys")

feature_store_timestamp_keys = dbutils.widgets.get("feature_store_timestamp_keys")


#### Read data from delta lake.

In [0]:
df = spark.read.format("delta").load(src_table)

#### Disable Delta Lake Optimization if Primary Key is String

- To improve query speed, Delta Lake on Databricks supports the ability to optimize the layout of data stored in cloud storage. Delta Lake on Databricks supports the algorithms called Z-Ordering.

- In ordinary delta lake table we externally had to apply these algorithms to optimize the delta tables. But in feature store delta table Z-ordering is implicitly applied on **primary_keys** of the table. 

- This Z-ordering is only applied on columns with **int** datatype.
- We disable Z-ordering in feature_table when its **primary_key** is of **string** data_type using the command *set spark.databricks.delta.optimize.zorder.checkStatsCollection.enabled = false*

In [0]:
primary_key_dtype = df.select(feature_store_primary_keys).dtypes

[('LOAN_ID', 'string')]


In [0]:
for key, key_data_type in primary_key_dtype:
    if key_data_type == 'string':
        spark.sql('set spark.databricks.delta.optimize.zorder.checkStatsCollection.enabled = false')

Hello


#### Feature Store Object

In [0]:
fs = feature_store.FeatureStoreClient()

#### Push Data to Feature Store

In [0]:
if spark.catalog.tableExists(feature_store_table):
    fs.write_table(
        name = feature_store_table,
        df = df,
        mode = 'merge'
    )
else:
    fs.create_table(
      name = feature_store_table,
      primary_keys = [feature_store_primary_keys],
      timestamp_keys = [feature_store_timestamp_keys],
      df = df,
      schema = df.schema
    )